# Seleção dinâmica de classificadores

***

Vamos precisar primeiramente instalar a biblioteca Deslib

(https://deslib.readthedocs.io/en/latest/index.html)

In [2]:
!pip install deslib

Defaulting to user installation because normal site-packages is not writeable


In [3]:
#importando as bibliotecas 

import pandas as pd

from sklearn.ensemble import BaggingClassifier
from deslib.static import Oracle, StaticSelection, SingleBest

from deslib.dcs import OLA, LCA
from deslib.des import KNORAU, KNORAE, METADES

from deslib.util.aggregation import majority_voting, average_combiner, minimum_combiner, maximum_combiner, product_combiner

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

## Conjunto de dados

In [4]:
# Breast Cancer Wisconsin (Diagnostic)
# https://archive.ics.uci.edu/
dataset = pd.read_csv('https://raw.githubusercontent.com/Francimaria/monitoria-ml/main/breast_cancer_wisconsin_diagnostic.csv')
dataset.head()

,diagnosis,radius_Mean,texture_Mean,periMeter_Mean,area_Mean,sMoothness_Mean,coMpactness_Mean,concavity_Mean,concave points_Mean,syMMetry_Mean,...,radius_worst,texture_worst,periMeter_worst,area_worst,sMoothness_worst,coMpactness_worst,concavity_worst,concave points_worst,syMMetry_worst,fractal_diMension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
#Mapeando os valores da classe para inteiro (para fins de visualização)
dataset["diagnosis"] = pd.factorize(dataset["diagnosis"])[0]
# M = 0 
# B = 1

In [6]:
X = dataset.drop([dataset.columns[0]], axis = 1)
y = dataset[dataset.columns[0]]

# separando o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# separando o conjunto de dados em treinamento e DSEL para as técnicas de DS
X_train, X_dsel, y_train, y_dsel = train_test_split(X_train, y_train, test_size=0.5, stratify=y_train, random_state=42)

# Seleção estática

Na seleção estática, a seleção dos classificadores é realizada na fase de treinamento.

O **Oracle** é um método abstrato que representa um esquema de seleção de classificador ideal.Ele sempre seleciona o classificador que previu o rótulo correto, para a amostra de consulta fornecida, se tal classificador existir.

In [7]:
# Pool de modelos
pool_classifiers = BaggingClassifier(n_estimators=2)
pool_classifiers.fit(X_train, y_train)

print(pool_classifiers.estimators_)

[DecisionTreeClassifier(random_state=1737885308), DecisionTreeClassifier(random_state=301752440)]


In [8]:
static_result = []
for n in [10, 20, 30]:
  pool_classifiers = BaggingClassifier(n_estimators=n)
  pool_classifiers.fit(X_train, y_train)

  static_model = StaticSelection(pool_classifiers)
  single_model = SingleBest(pool_classifiers)

  static_model.fit(X_dsel, y_dsel)
  single_model.fit(X_dsel, y_dsel)

  static_pred = static_model.predict(X_test)
  single_pred = static_model.predict(X_test)

  majority_pred = majority_voting(pool_classifiers, X_test)
  average_pred = average_combiner(pool_classifiers, X_test)
  product_pred = product_combiner(pool_classifiers, X_test)
  maximum_pred = maximum_combiner(pool_classifiers, X_test)
  minimum_pred = minimum_combiner(pool_classifiers, X_test)

  oracle = Oracle(pool_classifiers).fit(X_train, y_train)
  oracle_pred = oracle.predict(X_test, y_test)

  #calc f1-score for each model
  scores = [n,
            f1_score(y_test, static_pred),
            f1_score(y_test, single_pred),
            f1_score(y_test, majority_pred),
            f1_score(y_test, average_pred),
            f1_score(y_test, product_pred),
            f1_score(y_test, maximum_pred),
            f1_score(y_test, minimum_pred),
            f1_score(y_test, oracle_pred)]
  
  static_result.append(pd.DataFrame([scores], columns=['size_pool',
                                                       'Static',
                                                       'Single',
                                                       'Majority',
                                                       'Average',
                                                       'Product',
                                                       'Max',
                                                       'Min',
                                                       'Oracle']))

static_result = pd.concat(static_result).reset_index(drop=True).round(3)
static_result

,size_pool,Static,Single,Majority,Average,Product,Max,Min,Oracle
0,10,0.940,0.940,0.945,0.945,0.906,0.906,0.906,0.982
1,20,0.944,0.944,0.945,0.945,0.891,0.891,0.891,1.000
2,30,0.944,0.944,0.954,0.954,0.824,0.824,0.824,1.000


# Seleção dinâmica

A seleção dos classificadores é realizada para cada nova amostra de teste na fase de classificação.

Vamos avaliar diferentes abordagens:

1 - Seleção dinâmica de classificador 

2 - Seleção dinâmica de ensemble

META-DES: Framework de seleção de ensemble dinâmico. O META-DES usa meta-aprendizagem para fazer a seleção. São utilizados cinco   conjuntos distintos de metacaracterísticas, cada uma correspondendo a um critério diferente para medir o nível de competência de um classificador para a classificação de amostras de entrada.


Referência

Cruz, R. M., Sabourin, R., Cavalcanti, G. D., & Ren, T. I. (2015). META-DES: A dynamic ensemble selection framework using meta-learning. Pattern recognition, 48(5), 1925-1935.

In [9]:
dynamic_result = []
for n in [10, 20, 30]:
  pool_classifiers = BaggingClassifier(n_estimators=n)
  pool_classifiers.fit(X_train, y_train)

  # Dynamic classifier selection
  ola = OLA(pool_classifiers)
  lca = LCA(pool_classifiers)

  # Dynamic ensemble selection
  kne = KNORAE(pool_classifiers)
  knu = KNORAU(pool_classifiers)

  #meta-des
  meta = METADES(pool_classifiers)


  ola.fit(X_dsel, y_dsel)
  lca.fit(X_dsel, y_dsel)
  kne.fit(X_dsel, y_dsel)
  knu.fit(X_dsel, y_dsel)
  meta.fit(X_dsel, y_dsel)

  ola_pred = ola.predict(X_test)
  lca_pred = lca.predict(X_test)
  kne_pred = kne.predict(X_test)
  knu_pred = knu.predict(X_test)
  meta_pred = meta.predict(X_test)

  oracle = Oracle(pool_classifiers).fit(X_train, y_train)
  oracle_pred = oracle.predict(X_test, y_test)

  scores = [n,
            f1_score(y_test, ola_pred),
            f1_score(y_test, lca_pred),
            f1_score(y_test, kne_pred),
            f1_score(y_test, knu_pred),
            f1_score(y_test, meta_pred),
            f1_score(y_test, oracle_pred)]

  dynamic_result.append(pd.DataFrame([scores], columns=['size_pool', 'OLA', 'LCA', 'KNORAE', 'KNORAU','META-DES', 'Oracle']))

dynamic_result = pd.concat(dynamic_result).reset_index(drop=True).round(3)
dynamic_result

,size_pool,OLA,LCA,KNORAE,KNORAU,META-DES,Oracle
0,10,0.963,0.946,0.944,0.954,0.939,0.995
1,20,0.949,0.932,0.959,0.959,0.959,1.000
2,30,0.908,0.917,0.949,0.949,0.955,1.000


## E se quisermos usar um pool heterogênio

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [11]:
rng = 20

model_svc = SVC(probability=True, gamma='auto',
                random_state=rng).fit(X_train, y_train)
model_tree = DecisionTreeClassifier(random_state=rng,
                                    max_depth=10).fit(X_train, y_train)
model_knn = KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)

pool_classifiers = [model_svc, model_tree, model_knn]

ola = OLA(pool_classifiers)

ola.fit(X_dsel, y_dsel)

ola_pred = ola.predict(X_test)

print('F-score do OLA: %.3f' % f1_score(y_test, ola_pred))

F-score do OLA: 0.933
